# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 100 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

In [1]:
# Импорт
import pandas as pd
import numpy as np
from IPython.display import display, Markdown
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns

## Загрузка и подготовка данных

### Первый регион 

In [2]:
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_0.head(10)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
5,wX4Hy,0.969570,0.489775,-0.735383,64.741541
6,tL6pL,0.645075,0.530656,1.780266,49.055285
7,BYPU6,-0.400648,0.808337,-5.624670,72.943292
8,j9Oui,0.643105,-0.551583,2.372141,113.356160
9,OLuZU,2.173381,0.563698,9.441852,127.910945


In [3]:
data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [4]:
data_0.duplicated().sum()

0

### Второй регион

In [5]:
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_1.head(10)

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
5,HHckp,-3.327590,-2.205276,3.003647,84.038886
6,h5Ujo,-11.142655,-10.133399,4.002382,110.992147
7,muH9x,4.234715,-0.001354,2.004588,53.906522
8,YiRkx,13.355129,-0.332068,4.998647,134.766305
9,jG6Gi,1.069227,-11.025667,4.997844,137.945408


In [6]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [7]:
data_1.duplicated().sum()

0

### Третий регион

In [8]:
data_2 = pd.read_csv('/datasets/geo_data_2.csv')
data_2.head(10)

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
5,LzZXx,-0.758092,0.710691,2.585887,90.222465
6,WBHRv,-0.574891,0.317727,1.773745,45.641478
7,XO8fn,-1.906649,-2.458350,-0.177097,72.480640
8,ybmQ5,1.776292,-0.279356,3.004156,106.616832
9,OilcN,-1.214452,-0.439314,5.922514,52.954532


In [9]:
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [10]:
data_2.duplicated().sum()

0

### Вывод общий

Данные загружены в переменный, пропусков или неподходящих типо нет. Дублей также не имеется, проверять дубли без уникального идентификатора нет смысла, т.к. месторождения могут быть с похожими показателями.

## Обучение и проверка модели

Создадим функцию для обучения моделей всех 3х регионов.

In [11]:
def train_and_evaluate_model(data, region_name):
    # Разделение признаков и целевой переменной
    X = data.drop(['id', 'product'], axis=1)
    y = data['product']
    
    # Разделение на обучающую и валидационную выборки (75:25)
    X_train, X_valid, y_train, y_valid = train_test_split(
        X, y,
        test_size=0.25,
        random_state=12345
    )
    
    # Обучение модели линейной регрессии
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Предсказания на валидационной выборке
    predictions = model.predict(X_valid)
    
    # Расчет метрик
    rmse = np.sqrt(mean_squared_error(y_valid, predictions))
    mean_predicted = predictions.mean()
    
    # Сохранение результатов
    results = {
        'target_valid': y_valid,
        'predictions': predictions,
        'rmse': rmse,
        'mean_predicted': mean_predicted,
        'mean_actual': y_valid.mean()
    }
    
    print(f"Регион {region_name}:")
    print(f"    Средний запас предсказанного сырья: {mean_predicted:.2f} тыс. баррелей")
    print(f"    Средний запас фактического сырья: {y_valid.mean():.2f} тыс. баррелей")
    print(f"    RMSE модели: {rmse:.2f}")
    
    return results

In [12]:
results_0 = train_and_evaluate_model(data_0, 'Первый регион')
results_1 = train_and_evaluate_model(data_1, 'Второй регион')
results_2 = train_and_evaluate_model(data_2, 'Третий регион')

results_0, results_1, results_2

Регион Первый регион:
    Средний запас предсказанного сырья: 92.59 тыс. баррелей
    Средний запас фактического сырья: 92.08 тыс. баррелей
    RMSE модели: 37.58
Регион Второй регион:
    Средний запас предсказанного сырья: 68.73 тыс. баррелей
    Средний запас фактического сырья: 68.72 тыс. баррелей
    RMSE модели: 0.89
Регион Третий регион:
    Средний запас предсказанного сырья: 94.97 тыс. баррелей
    Средний запас фактического сырья: 94.88 тыс. баррелей
    RMSE модели: 40.03


({'target_valid': 71751     10.038645
  80493    114.551489
  2655     132.603635
  53233    169.072125
  91141    122.325180
              ...    
  12581    170.116726
  18456     93.632175
  73035    127.352259
  63834     99.782700
  43558    177.821022
  Name: product, Length: 25000, dtype: float64,
  'predictions': array([ 95.89495185,  77.57258261,  77.89263965, ...,  61.50983303,
         118.18039721, 118.16939229]),
  'rmse': 37.5794217150813,
  'mean_predicted': 92.59256778438035,
  'mean_actual': 92.07859674082927},
 {'target_valid': 71751     80.859783
  80493     53.906522
  2655      30.132364
  53233     53.906522
  91141      0.000000
              ...    
  12581    137.945408
  18456    110.992147
  73035    137.945408
  63834     84.038886
  43558     53.906522
  Name: product, Length: 25000, dtype: float64,
  'predictions': array([ 82.66331365,  54.43178616,  29.74875995, ..., 137.87934053,
          83.76196568,  53.95846638]),
  'rmse': 0.893099286775617,
  'mean

### Выводы

**Показатели**  
Первый регион:  
&nbsp;&nbsp;&nbsp;&nbsp;Средний запас предсказанного сырья: 92.59 тыс. баррелей  
&nbsp;&nbsp;&nbsp;&nbsp;Средний запас фактического сырья: 92.08 тыс. баррелей  
&nbsp;&nbsp;&nbsp;&nbsp;RMSE модели: 37.58

Второй регион:  
&nbsp;&nbsp;&nbsp;&nbsp;Средний запас предсказанного сырья: 68.73 тыс. баррелей  
&nbsp;&nbsp;&nbsp;&nbsp;Средний запас фактического сырья: 68.72 тыс. баррелей  
&nbsp;&nbsp;&nbsp;&nbsp;RMSE модели: 0.89

Третий регион:  
&nbsp;&nbsp;&nbsp;&nbsp;Средний запас предсказанного сырья: 94.97 тыс. баррелей  
&nbsp;&nbsp;&nbsp;&nbsp;Средний запас фактического сырья: 94.88 тыс. баррелей  
&nbsp;&nbsp;&nbsp;&nbsp;RMSE модели: 40.03

**Анализ**  
Во всех регионах среднее предсказанное значение близко к среднему фактическому:  
1: 92,59 против 92,08.  
2: 68,73 против 68,72.  
3: 94,97 против 94,88.  
Это указывает на отсутствие систематического смещения — модель не переоценивает и не недооценивает запасы в среднем. Это хороший знак, который говорит об объективности модели.

Наибольшие средние запасы наблюдаются в третьем регионе — около 94,9 тыс. баррелей. Затем идёт первый регион — около 92,1 тыс. баррелей. Наименьшие средние запасы — во втором регионе, около 68,7 тыс. баррелей. На первый взгляд, регион 3 выглядит наиболее привлекательным по объёму запасов.  
Однако в регионе 2 модель показывает исключительно высокую точность — RMSE составляет всего 0,89. Это означает, что модель почти идеально предсказывает объём нефти, а ошибка составляет менее 1000 баррелей, что является очень точным результатом.  
В регионах 1 и 3 RMSE значительно выше — около 37–40 тыс. баррелей. Это говорит о том, что модель работает, но с некоторой погрешностью.

**Итог**  
Основываясь на показателях качества моделей (RMSE), можно сделать вывод, что регион 2 является наиболее перспективным, несмотря на более низкие средние показатели запасов.  
Высокая точность модели позволяет снизить риски при принятии решений, что имеет решающее значение при инвестициях.

## Подготовка к расчёту прибыли

Укажем ключевые значения для рассчетов прибыли.

In [13]:
BUDGET = 10000000000  # Бюджет на разработку скважин в регионе
INCOME = 450000  # Доход с одной единицы (тыс. баррелей)
NUM_OF_WELLS = 200  # Количество скважин, которые будут разрабатываться

# Средние фактические запасы по регионам
mean_product_0 = data_0['product'].mean()
mean_product_1 = data_1['product'].mean()
mean_product_2 = data_2['product'].mean()

Рассчитаем достаточный объем сырья для безубыточной разработки новой скважины.

In [14]:
# Стоимость одной скважины
cost_per_well = BUDGET / NUM_OF_WELLS

# Минимальный объём сырья (в тыс. баррелей), необходимый для окупаемости одной скважины
min_volume_per_well = cost_per_well / INCOME
'Стоимость одной скважины:', cost_per_well, 'Минимальный объем сырья  на скважину для окупаемости:', min_volume_per_well
display(Markdown(f"""
Стоимость одной скважины: {cost_per_well},  
Минимальный объём для безубыточности: {min_volume_per_well:.2f} тыс. баррелей
"""))


Стоимость одной скважины: 50000000.0,  
Минимальный объём для безубыточности: 111.11 тыс. баррелей


Сравним со средним значением каждого региона.

In [15]:
display(Markdown(f"""
Сравнение средних запасов с порогом безубыточности ({min_volume_per_well:.2f} тыс. бр.)

| Регион | Средний запас | Выше порога? |
|--------|----------------|--------------|
| 1      | {mean_product_0:.2f} | {'Да' if mean_product_0 >= min_volume_per_well else 'Нет'} |
| 2      | {mean_product_1:.2f} | {'Да' if mean_product_1 >= min_volume_per_well else 'Нет'} |
| 3      | {mean_product_2:.2f} | {'Да' if mean_product_2 >= min_volume_per_well else 'Нет'} |
"""))


Сравнение средних запасов с порогом безубыточности (111.11 тыс. бр.)

| Регион | Средний запас | Выше порога? |
|--------|----------------|--------------|
| 1      | 92.50 | Нет |
| 2      | 68.83 | Нет |
| 3      | 95.00 | Нет |


### Вывод

Ключевые параметры:
- Бюджет на разработку скважин: 10 000 000 000 руб.
- Количество скважин для разработки: 200
- Доход с одной единицы (тыс. баррелей): 450 000 руб.
- Стоимость одной скважины: 50 000 000 руб.
- Минимальный объём сырья для окупаемости одной скважины: 111.11 тыс. баррелей

Ни один из регионов не достигает уровня безубыточности в среднем. Поскольку средние запасы ниже порога, прибыль возможна только при точном отборе лучших скважин с большим объемом.

## Расчёт прибыли и рисков 

In [16]:
def calculate_profit(target, predictions, top_wells=200):
    """
    Рассчитывает прибыль от разработки top_wells скважин, выбранных по наибольшим предсказаниям модели.
    
    Параметры:
    - target: фактические значения объёма
    - predictions: предсказания модели
    - top_wells: количество скважин для разработки (по умолчанию 200)
    
    Возвращает:
    - profit: прибыль в рублях
    - total_volume: суммарный объём нефти в выбранных скважинах (тыс. баррелей)
    """
    # Получаем индексы скважин с самыми высокими предсказаниями
    best_indices = np.argsort(predictions)[-top_wells:]
    
    # Суммируем фактический объём в этих скважинах
    total_volume = target.iloc[best_indices].sum()
    
    # Рассчитываем прибыль
    revenue = total_volume * INCOME
    profit = revenue - BUDGET
    
    return profit, total_volume

def bootstrap_analysis(target, predictions, n_bootstrap=1000, random_state=12345):
    """
    Проводит Bootstrap-анализ:
    - На каждой итерации случайно выбирает 500 скважин
    - Из них выбирает 200 с наибольшими предсказаниями
    - Считает прибыль по этим 200 скважинам
    
    Использует pandas.sample() для выборки 500 скважин.
    
    Параметры:
    - target: pd.Series, фактические значения объёма
    - predictions: array-like, предсказания модели
    - n_bootstrap: количество бутстрэп-выборок (по умолчанию 1000)
    - random_state: по умолчанию 12345
    
    Возвращает:
    - mean_profit: средняя прибыль
    - conf_interval: 95% доверительный интервал
    - loss_risk: доля убытков
    - profits: массив прибылей
    """
    # Преобразуем в Series с общим индексом
    data = pd.DataFrame({
        'target': target.reset_index(drop=True),
        'predictions': pd.Series(predictions).reset_index(drop=True)
    })
    
    state = np.random.RandomState(random_state)

    profits = []
    
    for i in range(n_bootstrap):
        seed = state.randint(0, 10000)
        
        # Случайно выбираем 500 скважин из валидационной выборки
        sampled_500 = data.sample(n=500, replace=False, random_state=seed)
        
        # Из 500 выбираем 200 с самыми высокими предсказаниями
        top_200 = sampled_500.nlargest(200, 'predictions')
        
        # Суммируем фактический объём (target) в этих 200 скважинах
        total_volume = top_200['target'].sum()
        
        # Рассчитываем прибыль
        revenue = total_volume * INCOME
        profit = revenue - BUDGET
        
        profits.append(profit)
    
    profits = np.array(profits)
    mean_profit = profits.mean()
    conf_interval = (np.percentile(profits, 2.5), np.percentile(profits, 97.5))
    loss_risk = (profits < 0).mean()
    
    return mean_profit, conf_interval, loss_risk, profits

In [17]:
profit_0, volume_0 = calculate_profit(results_0['target_valid'], results_0['predictions'])
profit_1, volume_1 = calculate_profit(results_1['target_valid'], results_1['predictions'])
profit_2, volume_2 = calculate_profit(results_2['target_valid'], results_2['predictions'])
display(Markdown(f"""
Прибыль при выборе 200 лучших скважин по предсказаниям модели

| Регион | Суммарный объём (тыс. бр) | Прибыль (млрд руб.) |
|--------|-------------------------------|----------------------|
| 1      | {volume_0:.1f}                | {profit_0 / 1e9:.3f} |
| 2      | {volume_1:.1f}                | {profit_1 / 1e9:.3f} |
| 3      | {volume_2:.1f}                | {profit_2 / 1e9:.3f} |

"""))


Прибыль при выборе 200 лучших скважин по предсказаниям модели

| Регион | Суммарный объём (тыс. бр) | Прибыль (млрд руб.) |
|--------|-------------------------------|----------------------|
| 1      | 29601.8                | 3.321 |
| 2      | 27589.1                | 2.415 |
| 3      | 28245.2                | 2.710 |



Оценим риски.

In [18]:
results_list = [results_0, results_1, results_2]

best_region = None
max_profit = -float('inf')
safe_regions = []

for i in range(3):
    res = results_list[i]
    mean_profit, conf_int, loss_risk, profits = bootstrap_analysis(
        res['target_valid'], res['predictions']
    )
    
    # Переводим в млрд для удобства
    mean_profit_bln = mean_profit / 1e9
    low_bln, high_bln = conf_int[0] / 1e9, conf_int[1] / 1e9
    
    print(f"Регион {i+1}:")
    print(f"  Средняя прибыль: {mean_profit_bln:.3f} млрд руб.")
    print(f"  95% доверительный интервал: ({low_bln:.3f}, {high_bln:.3f}) млрд руб.")
    print(f"  Риск убытков: {loss_risk:.1%}")
    
    # Проверяем критерий компании
    if loss_risk < 0.025:
        safe_regions.append(i)
        if mean_profit > max_profit:
            max_profit = mean_profit
            best_region = i
    
    print()

Регион 1:
  Средняя прибыль: 0.397 млрд руб.
  95% доверительный интервал: (-0.115, 0.915) млрд руб.
  Риск убытков: 6.1%

Регион 2:
  Средняя прибыль: 0.446 млрд руб.
  95% доверительный интервал: (0.034, 0.867) млрд руб.
  Риск убытков: 1.9%

Регион 3:
  Средняя прибыль: 0.386 млрд руб.
  95% доверительный интервал: (-0.195, 0.918) млрд руб.
  Риск убытков: 8.3%



## Вывод

Объеденим данные для наглядности:

| Параметр | Значение |
|---------|---------|
| Бюджет на разработку | 10 млрд руб. |
| Количество скважин к разработке | 200 |
| Доход с 1 тыс. баррелей | 450 000 руб. |
| Стоимость одной скважины | 50 000 000 руб. |
| Минимальный объём на скважину | 111.11 тыс. бр |

| Регион | Средний запас  | RMSE модели | Выше порога (111.11)? |
|--------|------------------------|------------|------------------------|
| 1     | 92.50 тыс. бр       | 37.58      |  Нет                 |
| 2     | 68.83 тыс. бр       | 0.89       | Нет                 |
| 3     | 95 тыс. бр          | 40.03      | Нет                 |

| Регион | Средняя прибыль | 95% доверительный интервал | Риск убытков |
|--------|------------------|-----------------------------|--------------|
| 1     | 0.397 млрд руб. | -0.115, 0.915 млрд руб.   | 6.1%         |
| 2     | 0.446 млрд руб. | 0.034, 0.867 млрд руб.   | 1.9%         |
| 3     | 0.386 млрд руб. | -0.195, 0.918 млрд руб.   | 8.3%         |

Итого средние запасы в 3 регионе выше (95 тыс. бр), далее 1 регион (92,5 тыс. бр) и 2 (68,83 тыс. бр). Минимальный объем на скважину (111,11 тыс. бр) не достигает ни один регион. Значит выбирать нужно не по среднему, а смотреть 200 лучших скважин и будет очень важно уметь выбрать эти скважины.

По качеству модели выигрывает 2 регион, там модель почти идеальна (0,89), далее у нас идет 1 регион (37,58) и 3 регион (40,03). Чем точнее модель, тем надежнее предсказания и ранжирование скважин.

Прибыль в 1 и 3 регионах показывает и убыток, а риск убытков не проходит по критерию компании (< 2,5%). Второй регион не уходит в убыток и средняя прибыль выше остальных регионов.

С учетом всего выявленного рекомендую взять 2 регион, так как:
1. Это единственный регион, проходящий по критерию % риска. Тогда как 1 и 3 регионы не проходят по данному критерию (6,1% и 8,3% соответственно).
2. Данный регион принесет в среднем больше прибыли, чем 1 и 3 регионы.
3. Самая точная модель (RMSE = 0,89), которая подскажет, какие скважины стоит взять в работу.  
Данный регион будет наиболее безопасным и прибыльным выбором.

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.